In [9]:
from llmfactor import LLMFactorAnalyzer, LoggerSingleton
import re

In [10]:
analyzer = LLMFactorAnalyzer(
    base_url="http://localhost:5678/v1",
    api_key="token-abc123",
    model="./models/llama-3.1-8B-instruct-Q8_0.gguf",
    logger=LoggerSingleton.get_logger())
client = analyzer.client

In [11]:
tickers = analyzer.get_available_tickers()
ticker = tickers[0]
stock_target = ticker

factor_k = 5
price_k = 5

dates = analyzer.get_available_dates(ticker, price_k)
target_date = dates[8]

news_str = analyzer.news_data.get_news_by_date(ticker, target_date)
price_movements = analyzer.price_data.get_price_movements(ticker, target_date, price_k)
price_str, price_str_last = analyzer.format_price_movements(price_movements, ticker, target_date)

In [12]:
print(price_str + price_str_last)

On 2018-01-16, the stock price of AAPL fell.
On 2018-01-17, the stock price of AAPL rose.
On 2018-01-18, the stock price of AAPL rose.
On 2018-01-19, the stock price of AAPL fell.
On 2018-01-22, the stock price of AAPL


In [13]:
print(*price_movements, sep="\n")

{'date': Timestamp('2018-01-16 00:00:00'), 'rise': False}
{'date': Timestamp('2018-01-17 00:00:00'), 'rise': True}
{'date': Timestamp('2018-01-18 00:00:00'), 'rise': True}
{'date': Timestamp('2018-01-19 00:00:00'), 'rise': False}
{'date': Timestamp('2018-01-22 00:00:00'), 'rise': False}


In [14]:
print(len(news_str))

19793


In [15]:
def extract_factors(text):
    # Pattern to match numbered items with their descriptions
    # Looks for: number, dot, colon, and remaining text
    pattern = r'\d+\.[^:]+:\s+([^\n]+)'

    # Find all matches in the text
    matches = re.finditer(pattern, text)

    # Create a dictionary to store factors with their descriptions
    result_str = ""

    for match in matches:
        result_str += match.group(0) + "\n"

    return result_str[:-1]

In [16]:
factor_extraction = analyzer.client.chat.completions.create(
    model=analyzer.model,
    temperature=0,
    max_tokens=factor_k * 200,
    logprobs=5,
    messages=[
        {"role": "system", "content": f"Please extract the top {factor_k} factors that may affect the stock price of {ticker} from the following news."},
        {"role": "user", "content": "# news list\n\n" + news_str},
        {"role": "user", "content": f"Now, tell me. What are the {factor_k} most influential market drivers for {ticker} based on recent news? Write your answer with following format:\n" + "".join([f"\n{i}." for i in range(1, factor_k + 1)])},
    ])

factor_str = factor_extraction.choices[0].message.content
factors = extract_factors(factor_str)

print("```")
print(factor_str)
print("```")
print()
print("```")
print(factors)
print("```")

```
Based on recent news, here are the 5 most influential market drivers for AAPL:

1. **iPhone X Production Cuts**: Speculation that Apple may cut back production of the high-end iPhone X has been weighing on the stock. This has led to a sell-off in optical component makers such as Lumentum, Finisar, and II-VI.
2. **Tax Reform**: Apple's plans to bring back the $250 billion in cash reserves it holds overseas is expected to benefit stakeholders, but the company's tax strategy and potential changes to its cash repatriation plans are being closely watched.
3. **HomePod and Smart Speaker Market**: Apple's HomePod, a smart speaker integrated with Siri, is expected to hit the shelves in February. The company's entry into the smart speaker market is seen as a key opportunity for growth, but it also faces stiff competition from Amazon and Google.
4. **Education Initiatives**: Apple's partnership with Malala Yousafzai to educate 100,000 girls and its plans to support global education initiativ

In [ ]:
factor_inverse = analyzer.client.chat.completions.create(
    model=analyzer.model,
    temperature=0,
    messages=[
        {"role": "system",
         "content": f"You are a playful assistant, saying opposite situation of the given statement by user. Your goal is to write the opposite situation of each given statement."},
        {"role": "user", "content": factors}
    ]
)

'''
 "content": f"You are a playful assistant, saying opposite situation of the given statement by user. Your goal is to sneakily blend the opposite situation of the given statement."
'''

factor_str = factor_inverse.choices[0].message.content
factors = extract_factors(factor_str)

print("```")
print(factor_str)
print("```")
print()
print("```")
print(factors)
print("```")

In [18]:
answer_extraction = analyzer.client.chat.completions.create(
    model=analyzer.model,
    temperature=0,
    messages=[
        {"role": "system", "content": "Based on the following information, please judge the direction of the stock price from rise/fall, fill in the blank and give reasons."},
        {"role": "system", "content": f"These are the main factors that may affect this stock’s price recently:\n{factors}."},
        {"role": "system", "content": price_str},
        {"role": "assistant", "content": price_str_last},
    ]
)

answer = answer_extraction.choices[0].message.content
print(answer)

Based on the given information, I would predict that the stock price of AAPL on 2018-01-22 will **fall**.

Reasons:

* The recent downgrades of Apple's suppliers, such as Lumentum and Finisar, due to concerns over iPhone X production cuts and component costs, are likely to continue to weigh on the stock.
* The uncertainty surrounding Apple's tax strategy and potential changes to its cash repatriation plans may also contribute to a decline in the stock price.
* The competition in the smart speaker market, where Apple's HomePod is expected to launch, may also lead to a decline in the stock price if the product does not meet expectations.

Overall, the negative factors mentioned in the information seem to outweigh the positive ones, leading me to predict a decline in the stock price.
